# Goal

The goal is to take in a series of inputs for each player available for purchase in FPL -> and turn that into a prediction for their points for the gameweek. 

# What is needed?

In order to generate an expected point value for a player, we need data about players and what they scored each week. <br>

It does not seem like this sort of information is being saved anywhere. As such, the first phase of this project will be setting up the pipeline to collect this data each gameweek. We will want to collect a bunch of information from a few different sources, things like percentage of minutes played, xG Per 90, xA per 90, "threat", "influence, "creativity (those 3 being FPL generated metrics), opposition xG conceded, home or away, etc. <br>

We will want to be able to collect this weekly as a snapshot BEFORE the matches are played. After they are played, we will append a "points_scored" to each record. Eventually we aim to be able to predict this points scored value give all the data we collect, but we need the data in the week-by-week format in order to do this. 

# Phase 1: Week-by-week Historical Data Collection

## 1) Data sources and desired attributes

Here I will outline the specific data sources I am going to pull from, and what data I want. 

### Fbref

Think of this site as providing data from two perspectives: team and individual. <br>

As for team data, we want to have attributes that give an idea of how the individual's team is performing, but also how the team they are playing against is performing. Therefore..

- all expected stats per 90 minutes FOR (don't even pull goals and assists, I just care about expected). We will use this to see how good of an attacking team this player is playing for, and how bad of an attacking team they are playing against
- all expected stats per 90 minutes AGAINST (tells us how good or bad of a defense this player plays for or is up against)

And for the individual perspective:

- percentage of minutes played this season - "min%" (is the player playing a lot?)
- expected stats per 90 (how effective is this player attacking-wise?)
- tackle + challenge + blocks, per 90 data (how effective is this player defensively?)
- yellow/ red cards per 90 (these actions lose points, so we want to know about them)
- penalty share, a number between 0 and 1 (we want to know if a player is their team's penalty kick taker, as this is a good way to get points)

We will also get all the scheduling information out of this site. 

### Official fantasy premier league site

We want to know some stuff as it relates to the game itself. These include:

- price and selection %, won't really assist in predicting points (or rather we don't want to use them for that) but will come in handy for later functionality with the model, like picking differentials and building a squad
- FORM - very important. We want to know how this player is performing coming into the gameweek
- finally, actual points scored.

Remember, these are all snapshot statistics - we want to know what these values were before the gameweek, and after the gameweek, we want to append the points scored to each record. 

### Proposed workflow

1) A script runs to start to fill out the games to be played in the next gameweek. It fills in a record for each player, with the gameweek, individual's team, and opposition.

2) We then access the Fbref data source in order to get team and opposition data. Basically, we will match on the player's team first, getting expected data both for and against - then we repeat the process for the opposition.

3) Now, we have the player, who they are playing, and data about how their team is performing per 90 and how their opposition is performing per 90 up to this point in the season. We should now attach all the data from the player perspective to each row. Get all the per 90 data. This should all be quite simple except for the penalty kick share, which will require a simple calculation to see what perfecntage of a team's penalty kicks the player has taken.

4) Now, join in the data from the official FPL website. Match based on player name, and grab price, % selection, form, and the column "points_scored" but leave this BLANK (we will not know it at the time this script runs).

5) We will let the game week happen, then run the script that gets player points for the week from the official FPL site. Join this in based on player name to the records we just created, using matchweek and player name as the combined key. 

In [1]:
import soccerdata as sd
import pandas as pd
from datetime import datetime
import requests
from thefuzz import process
import warnings

curr_gameweek = 12

warnings.simplefilter(action='ignore', category=FutureWarning)

def get_fixtures(week_wanted):
    """
    grabs the list of games for the week, extracts only the cleaned team names of home and away team, as well as match_week, 
    """
    fbref = sd.FBref(leagues='ENG-Premier League', seasons='2025-2026')
    schedule = fbref.read_schedule()
    schedule['date'] = pd.to_datetime(schedule['date'], errors='coerce')
    schedule = schedule[schedule['week'] == week_wanted]

    return schedule[['home_team','away_team','week']]


def get_fbref_player_stats(season='2025-2026',pt_threshold=40):
    """
    grabs all player individual statistics that we want
    """
    fbref = sd.FBref('ENG-Premier League', season)

    standard = fbref.read_player_season_stats(stat_type="standard")
    shooting = fbref.read_player_season_stats(stat_type="shooting")
    passing = fbref.read_player_season_stats(stat_type="passing")
    defense = fbref.read_player_season_stats(stat_type="defense")
    playing_time = fbref.read_player_season_stats(stat_type="playing_time")

    def flatten_cols(df):
        df = df.copy()
        df.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in df.columns.values]
        return df

    standard = flatten_cols(standard)
    shooting = flatten_cols(shooting)
    passing = flatten_cols(passing)
    defense = flatten_cols(defense)
    playing_time = flatten_cols(playing_time)

    for df in [standard, shooting, passing, defense, playing_time]:
        df.reset_index(inplace=True)
        df.rename(columns={'index': 'player'}, inplace=True)

    metadata_cols = ['season', 'league', 'team', 'nation_', 'pos_', 'age_', 'born_']
    for df in [standard, shooting, passing, defense]:
        df.drop(columns=[c for c in metadata_cols if c in df.columns], inplace=True)

    fbref_stats = standard
    for df in [shooting, passing, defense, playing_time]:
        fbref_stats = fbref_stats.merge(df, on='player', how='outer')

    
    fbref_stats['Tackles_Tkl_per90'] = fbref_stats['Tackles_Tkl'] / fbref_stats['Playing Time_90s_y']
    fbref_stats['Blocks_Blocks_per90'] = fbref_stats['Blocks_Blocks'] / fbref_stats['Playing Time_90s_y']
    fbref_stats['yellow_per90'] = fbref_stats['Performance_CrdY'] / fbref_stats['Playing Time_90s_y']
    fbref_stats['red_per90'] = fbref_stats['Performance_CrdR'] / fbref_stats['Playing Time_90s_y']

    fbref_stats = fbref_stats[fbref_stats['Playing Time_Min%'] >= pt_threshold]

    name_map = {
        "Alisson": "Alisson Becker",
        "André": "André Trindade da Costa Neto",
        "Benjamin Šeško": "Benjamin Sesko",
        "Bernardo Silva": "Bernardo Mota Veiga de Carvalho e Silva",
        "Beto": "Norberto Bercique Gomes Betuncal",
        "Bruno Guimarães": "Bruno Guimarães Rodriguez Moura",
        "Casemiro": "Carlos Henrique Casimiro",
        "David Raya": "David Raya Martín",
        "Diego Gómez": "Diego Gómez Amarilla",
        "Diogo Dalot": "Diogo Dalot Teixeira",
        "Emi Buendía": "Emiliano Buendía Stati",
        "Evanilson": "Francisco Evanilson de Lima Barbosa",
        "Ezri Konsa": "Ezri Konsa Ngoyo",
        "Ferdi Kadioglu": "Ferdi Kadıoğlu",
        "Florentino Luís": "Florentino Ibrain Morris Luís",
        "Gabriel Magalhães": "Gabriel dos Santos Magalhães",
        "Hugo Bueno": "Hugo Bueno López",
        "Jeremy Doku": "Jérémy Doku",
        "Joelinton": "Joelinton Cássio Apolinário de Lira",
        "Joshua King": "Josh King",
        "João Gomes": "Gustavo Nunes Fernandes Gomes",
        "João Palhinha": "João Maria Lobo Alves Palhares Costa Palhinha Gonçalves",
        "João Pedro": "João Pedro Junqueira de Jesus",
        "Lucas Paquetá": "Lucas Tolentino Coelho de Lima", 
        "Lucas Perri": "Lucas Estella Perri",
        "Marc Cucurella": "Marc Cucurella Saseta",
        "Mateus Fernandes": "Mateus Gonçalo Espanha Fernandes",
        "Matheus Cunha": "Matheus Santos Carneiro da Cunha",
        "Max Kilman": "Maximilian Kilman",
        "Moisés Caicedo": "Moisés Caicedo Corozo",
        "Morato": "Felipe Rodrigues Da Silva",
        "Murillo": "Murillo Costa dos Santos",
        "Nicolás González": "Nico González Iglesias",
        "Pedro Neto": "Pedro Lomba Neto",
        "Pedro Porro": "Pedro Porro Sauceda",
        "Raúl Jiménez": "Raúl Jiménez Rodríguez",
        "Richarlison": "Richarlison de Andrade",
        "Rúben Dias": "Rúben dos Santos Gato Alves Dias",
        "Santiago Bueno": "Santiago Ignacio Bueno",
        "Thiago": "Igor Thiago Nascimento Rodrigues",
        "Valentino Livramento": "Tino Livramento",
        "Yeremi Pino": "Yéremy Pino Santos",
        "Álex Jiménez": "Álex Jiménez Sánchez"
    }

    fbref_stats['player'] = fbref_stats['player'].apply(lambda x: name_map.get(x, x))

    return fbref_stats[['player','Playing Time_Min%','Per 90 Minutes_xG','Per 90 Minutes_xAG','Tackles_Tkl_per90','Blocks_Blocks_per90','yellow_per90','red_per90']]
    
def get_teams(season="2025-2026"):
    fbref = sd.FBref("ENG-Premier League", season)
    
    teams = fbref.read_team_season_stats(stat_type="shooting").index.tolist()
    
    data = []
    today = pd.Timestamp(datetime.today().date())
    
    for team in teams:
        matches = fbref.read_team_match_stats(stat_type="schedule", team=team)
        
        matches = matches[pd.to_datetime(matches["date"]) <= today]
        
        if matches.empty:
            continue
        
        xG_for_per90 = matches["xG"].sum() / len(matches)
        xG_against_per90 = matches["xGA"].sum() / len(matches)
        
        data.append({
            "team": team,
            "xG_for_per90": xG_for_per90,
            "xG_against_per90": xG_against_per90
        })

    df = pd.DataFrame(data)
    
    df['team'] = df['team'].astype(str).str.replace(r'^.*,\s*(.*?)\)$', r'\1', regex=True)

    df['team'] = df['team'].str.replace("'", "").str.strip()

    name_map = {
    "Manchester Utd": "Man Utd",
    "Manchester City": "Man City",
    "Tottenham": "Spurs",
    "Nott'ham Forest": "Nott'm Forest",
    "Newcastle Utd": "Newcastle",
    "Leeds United" : "Leeds"}
    
    df['team_name'] = df['team'].replace(name_map)

    return df


def get_players():
    """
    Grabs a list of all FPL players
    """
    url = "https://fantasy.premierleague.com/api/bootstrap-static/"
    response = requests.get(url)
    data = response.json()
    
    players = pd.DataFrame(data['elements'])
    teams = {team['id']: team['name'] for team in data['teams']}
    players['team_name'] = players['team'].map(teams)
    
    positions = {pos['id']: pos['singular_name'] for pos in data['element_types']}
    players['position'] = players['element_type'].map(positions)
    
    players_df = players[['id', 'first_name', 'second_name', 'team_name', 'position', 'now_cost']].copy()
    players_df['full_name'] = players_df['first_name'] + " " + players_df['second_name']
    
    return players_df


def fuzzy_match(fpl_df, fbref_df, threshold=92):
    """
    Fuzzy matches FPL players (already mapped) to FBref player stats by name
    """
    
    fbref_names = fbref_df['player'].tolist()
    
    fpl_names = fpl_df['full_name'].tolist()

    mapping = {}
    for name in fpl_names:
        if pd.isna(name):
            mapping[name] = None
            continue
        match, score = process.extractOne(name, fbref_names)
        mapping[name] = match if score >= threshold else None

    fpl_df['matched_fbref'] = fpl_df['full_name'].map(mapping)

    merged = fpl_df.merge(fbref_df, left_on='matched_fbref', right_on='player', how='inner')

    merged = merged.drop_duplicates(subset=['player'])

    return merged

import pandas as pd
import requests

def get_fpl_table():
    """Pulls current PL standings from the official FPL API."""
    url = "https://fantasy.premierleague.com/api/bootstrap-static/"
    data = requests.get(url).json()

    teams = pd.DataFrame(data["teams"])[["name", "short_name", "position"]]

    # Normalize names to match your naming scheme
    name_map = {
        "Manchester United": "Man Utd",
        "Manchester City": "Man City",
        "Tottenham Hotspur": "Spurs",
        "Nottingham Forest": "Nott'm Forest",
        "Newcastle United": "Newcastle",
        "Leeds United": "Leeds",
        "Brighton and Hove Albion": "Brighton",
        "Wolverhampton Wanderers": "Wolves",
        "West Ham United": "West Ham",
        "Aston Villa": "Aston Villa",
        "Sheffield United": "Sheffield Utd",
    }

    teams["team_name"] = teams["name"].replace(name_map)

    return teams[["team_name", "position"]]


def get_fixtures_and_league_spots(gameweek=curr_gameweek):
    fixtures = get_fixtures(gameweek)

    name_map = {
        "Manchester Utd": "Man Utd",
        "Manchester City": "Man City",
        "Tottenham": "Spurs",
        "Nott'ham Forest": "Nott'm Forest",
        "Newcastle Utd": "Newcastle",
        "Leeds United": "Leeds"
    }

    fixtures['home_team'] = fixtures['home_team'].replace(name_map)
    fixtures['away_team'] = fixtures['away_team'].replace(name_map)

    home_df = fixtures[["home_team", "week"]].rename(columns={"home_team": "team"})
    home_df["home"] = 1

    away_df = fixtures[["away_team", "week"]].rename(columns={"away_team": "team"})
    away_df["home"] = 0

    combined = pd.concat([home_df, away_df], ignore_index=True)

    standings = get_fpl_table()

    final = combined.merge(standings, left_on="team", right_on="team_name", how="left")
    final = final.drop(columns=["team_name"])

    return final.sort_values("position")


def join_it_all_together():
    df_fpl = get_players()
    df_fbref = get_fbref_player_stats()
    df_teams = get_teams()
    df_fuz = fuzzy_match(df_fpl,df_fbref)
    df = df_fuz.merge(df_teams,left_on="team_name",right_on="team",how="left")

    df_fix = get_fixtures_and_league_spots() 

    df = df.merge(df_fix,on='team',how='left')

    df = df.rename(columns={'team':'team_name','position_x':'player_position','now_cost':'current_fpl_cost',
                        'Playing Time_Min%':'playing_time_min_percentage','Per 90 Minutes_xG':'xg_per_90',
                       'Per 90 Minutes_xAG':'xag_per_90','Tackles_Tkl_per90':'tackles_per_90','Blocks_Blocks_per90':'blocks_per_90',
                       'yellow_per90':'yellows_per_90','red_per90':'reds_per_90','xG_for_per90':'team_xg_per_90',
                       'xG_against_per90':'team_xg_against_per_90','week':'gameweek','home':'is_at_home',
                       'position_y':'team_league_position'})
    
    return df[['full_name','team_name','player_position','current_fpl_cost','playing_time_min_percentage','xg_per_90','xag_per_90','blocks_per_90',
          'yellows_per_90','reds_per_90','team_xg_per_90','team_xg_against_per_90','gameweek','is_at_home','team_league_position']]


[11/18/25 14:12:01] INFO     No custom team name replacements found. You can configure these in       ]8;id=926705;file:///home/tars/jupyter/lib/python3.12/site-packages/soccerdata/_config.py\_config.py]8;;\:]8;id=800810;file:///home/tars/jupyter/lib/python3.12/site-packages/soccerdata/_config.py#91\91]8;;\
                             /home/tars/soccerdata/config/teamname_replacements.json.                              

                    INFO     No custom league dict found. You can configure additional leagues in    ]8;id=192676;file:///home/tars/jupyter/lib/python3.12/site-packages/soccerdata/_config.py\_config.py]8;;\:]8;id=191355;file:///home/tars/jupyter/lib/python3.12/site-packages/soccerdata/_config.py#197\197]8;;\
                             /home/tars/soccerdata/config/league_dict.json.                                        

In [5]:
df = join_it_all_together()
df.shape

[11/18/25 14:22:36] INFO     Saving cached data to /home/tars/soccerdata/data/FBref                  ]8;id=628265;file:///home/tars/jupyter/lib/python3.12/site-packages/soccerdata/_common.py\_common.py]8;;\:]8;id=570057;file:///home/tars/jupyter/lib/python3.12/site-packages/soccerdata/_common.py#263\263]8;;\

[11/18/25 14:22:38] INFO     Saving cached data to /home/tars/soccerdata/data/FBref                  ]8;id=261510;file:///home/tars/jupyter/lib/python3.12/site-packages/soccerdata/_common.py\_common.py]8;;\:]8;id=201588;file:///home/tars/jupyter/lib/python3.12/site-packages/soccerdata/_common.py#263\263]8;;\

[11/18/25 14:24:44] INFO     Saving cached data to /home/tars/soccerdata/data/FBref                  ]8;id=864282;file:///home/tars/jupyter/lib/python3.12/site-packages/soccerdata/_common.py\_common.py]8;;\:]8;id=869445;file:///home/tars/jupyter/lib/python3.12/site-packages/soccerdata/_common.py#263\263]8;;\

(246, 15)

In [ ]:
# all that is left todo.. 

# 1) A single method that is meant to be ran after the gameweek (the above is an before) that grabs the points scored for each player and associates
#    them together

# 2) Save this a SQlite db or Postgres db.. something running on the server. Save the .db file in the github too.. for extra safety. 

In [15]:
def get_players_with_points(gameweek):
    """
    Returns FPL players + their total points for a specific gameweek.
    """
    url = "https://fantasy.premierleague.com/api/bootstrap-static/"
    response = requests.get(url)
    data = response.json()

    # Base players df
    players = pd.DataFrame(data['elements'])
    teams = {team['id']: team['name'] for team in data['teams']}
    players['team_name'] = players['team'].map(teams)

    positions = {pos['id']: pos['singular_name'] for pos in data['element_types']}
    players['position'] = players['element_type'].map(positions)

    # Build base DF
    players_df = players[['id', 'first_name', 'second_name', 'team_name', 'position', 'now_cost']].copy()
    players_df['full_name'] = players_df['first_name'] + " " + players_df['second_name']

    # Get points for each player in the given gameweek
    points_list = []
    for pid in players_df['id']:
        p_url = f"https://fantasy.premierleague.com/api/element-summary/{pid}/"
        p_data = requests.get(p_url).json()

        history = p_data.get("history", [])
        gw_record = next((gw for gw in history if gw["round"] == gameweek), None)

        points_list.append(gw_record["total_points"] if gw_record else 0)

    players_df["gw_points"] = points_list
    
    return players_df[['full_name','gw_points']]

df_ = get_players_with_points(11)

In [17]:
df_.head(30)

,full_name,gw_points
0,David Raya Martín,1
1,Kepa Arrizabalaga Revuelta,0
2,Karl Hein,0
3,Tommy Setford,0
4,Gabriel dos Santos Magalhães,1
5,William Saliba,1
6,Riccardo Calafiori,1
7,Jurriën Timber,1
8,Jakub Kiwior,0
9,Myles Lewis-Skelly,0


In [20]:
X = pd.read_csv("/home/tars/Projects/fpl-oracle/X_12.csv")
filtered = df_[df_['full_name'].isin(X['full_name'])]
filtered.shape

(246, 2)

# Phase 2: Model

- we will only start building this after 3 or 4 weeks of data collection
- Some things to keep in mind:
  - do we want to exclude from training all instances where a player scored 0 points? I feel this is a good way to get rid of instances where players just did not play. Even if a player plays and gets 0 points, this is a rare instance. Usually they will at least get 1.

In [1]:
# grab our data, pairing X to y for each game-week

import os
import re
import pandas as pd

DATA_DIR = "/home/tars/Projects/fpl-oracle/data"

files = os.listdir(DATA_DIR)

pattern = re.compile(r'^(X|y)_(\d+)\.csv$')

file_map = {}

for f in files:
    match = pattern.match(f)
    if match:
        kind, num = match.groups()
        file_map.setdefault(num, {})[kind] = f

merged_dfs = []

for num, pair in file_map.items():
    if 'X' in pair and 'y' in pair:
        x_path = os.path.join(DATA_DIR, pair['X'])
        y_path = os.path.join(DATA_DIR, pair['y'])

        X = pd.read_csv(x_path)
        y = pd.read_csv(y_path)

        merged = X.merge(
            y,
            on="full_name",
            how="inner"   
        )

        merged["source_id"] = int(num)

        merged_dfs.append(merged)

final_df = pd.concat(merged_dfs, ignore_index=True)

print(final_df.shape)

(1230, 17)


In [26]:
df = final_df.copy()

df = df[df['gw_points'] != 0]

df.head()

,full_name,team_name,player_position,current_fpl_cost,playing_time_min_percentage,xg_per_90,xag_per_90,blocks_per_90,yellows_per_90,reds_per_90,team_xg_per_90,team_xg_against_per_90,gameweek,is_at_home,team_league_position,gw_points,source_id
0,David Raya Martín,Arsenal,Goalkeeper,59,100.0,0.00,0.00,0.000000,0.090909,0.0,1.685714,0.514286,13.0,0.0,1.0,3,13
4,Jurriën Timber,Arsenal,Defender,64,89.1,0.20,0.06,0.918367,0.204082,0.0,1.685714,0.514286,13.0,0.0,1.0,2,13
5,Bukayo Saka,Arsenal,Midfielder,101,68.8,0.32,0.26,1.315789,0.000000,0.0,1.685714,0.514286,13.0,0.0,1.0,5,13
7,Declan Rice,Arsenal,Midfielder,71,90.3,0.10,0.17,0.808081,0.000000,0.0,1.685714,0.514286,13.0,0.0,1.0,4,13
8,Mikel Merino Zazón,Arsenal,Midfielder,60,42.2,0.15,0.08,1.304348,0.000000,0.0,1.685714,0.514286,13.0,0.0,1.0,9,13


In [44]:
# I don't think there is a linear relationship between any of this information and gameweek points.

# Let's try a tree-based model first. 

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import HistGradientBoostingRegressor

target = "gw_points"

X = final_df.drop(columns=[target, "full_name","source_id"])
y = final_df[target]

categorical = ["team_name", "player_position"]
numeric = X.columns.difference(categorical)

preprocessor = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical),
        ("num", "passthrough", numeric),
    ]
)

train_idx = final_df["gameweek"] <= 17
test_idx = final_df["gameweek"] > 17

X_train, X_test = X[train_idx], X[test_idx]
y_train, y_test = y[train_idx], y[test_idx]

model = Pipeline(
    steps=[
        ("prep", preprocessor),
        ("gb", HistGradientBoostingRegressor(
            max_depth=6,
            learning_rate=0.05,
            max_iter=300,
            random_state=42
        )),
    ]
)

model.fit(X_train, y_train)

,steps,"[('prep', ...), ('gb', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('cat', ...), ('num', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [49]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
from scipy.stats import spearmanr

pred = model.predict(X_test)

mae = mean_absolute_error(y_test, pred)
rmse = np.sqrt(mean_squared_error(y_test, pred))
r2 = r2_score(y_test, pred)

print(f"MAE  : {mae:.3f}")
print(f"RMSE : {rmse:.3f}")
print(f"R²   : {r2:.3f}")
print(f"Spearman rank correlation: {spearman.correlation:.3f}")

MAE  : 2.222
RMSE : 3.345
R²   : -0.245
Spearman rank correlation: 0.313


In [58]:
# Alright, this model is fine for now. It will get better with more data and I can play around with parameters later

# Next, I need to optimize the process in which we run the scripts

# 1) Need a way to monitor when the gameweek has changed. Each day, maybe I grab the what the next game's gameweek is from. If it changes, we run step 2
# 2) We can actually run a single script. First, run the after_gameweek to get the next gameweek - 1 gameweek's statistics for who scored what.. then run the before gameweek for the next
#    gameweek
# 3) When this script runs, we can also now run our model script.. get a new predictions file/ append the latest predictions to the file, and send that to the website directory


In [2]:
import requests
import pandas as pd
from pathlib import Path

def get_next_gameweek():
    """
    Returns the next FPL gameweek number based on current date.
    """
    url = "https://fantasy.premierleague.com/api/bootstrap-static/"
    data = requests.get(url).json()

    events = pd.DataFrame(data["events"])

    next_gw = events.loc[events["is_next"], "id"]

    if next_gw.empty:
        raise ValueError("No upcoming gameweek found")

    return int(next_gw.iloc[0])

get_next_gameweek()

19